In [1]:
import pandas as pd
import nltk
import re

from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df=pd.read_csv(r"../input/amazon-music-reviews/Musical_instruments_reviews.csv")

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [4]:
df_use=df[['summary']]

In [5]:
result = [] 
for value in df["overall"]: 
    if value == 1.0: 
        result.append("Negative") 
    elif value==2.0: 
        result.append("Negative") 
    elif value==3.0: 
        result.append("Neutral")
    elif value==4.0: 
        result.append("Positive")
    elif value==5.0: 
        result.append("Positive")
       
df_use["Result"] = result    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
df_use.shape

(10261, 2)

In [7]:
#df_use=df_use.loc[df_use['Result']!='Neutral']

In [8]:
df_use.groupby('Result').describe()

summary                   
           count unique    top freq
Result                             
Negative     467    457   Crap    3
Neutral      772    740     OK    5
Positive    9022   7702  Great   42

In [9]:
df_use.head()

,summary,Result
0,good,Positive
1,Jake,Positive
2,It Does The Job Well,Positive
3,GOOD WINDSCREEN FOR THE MONEY,Positive
4,No more pops when I record my vocals.,Positive


In [10]:
df_use.shape

(10261, 2)

In [11]:
df_use.tail(30)

,summary,Result
10231,Cadillac of Guitar Strings,Positive
10232,Great for pickin' on the Tele or almost anythi...,Positive
10233,All they claim to be,Positive
10234,Nice strings,Positive
10235,Great strings! They really sing!,Positive
10236,"A little skinny for me, but the quality is und...",Positive
10237,Noticeable Improvement... but at what cost?,Neutral
10238,Nice bright sound with easy action,Positive
10239,Stay In Tune Better,Positive
10240,A Good Guitar Deserves Good Strings,Positive


In [12]:
train_df=df_use.iloc[0:-1]

In [13]:
train_df.shape

(10260, 2)

In [14]:
train_df.tail()

,summary,Result
10255,"Hmmm.... I like them, but with a lot of reserv...",Neutral
10256,Five Stars,Positive
10257,"Long life, and for some players, a good econom...",Positive
10258,Good for coated.,Positive
10259,Taylor Made,Positive


In [15]:
#test_df=df_df.iloc[[-1]]
#test_df.shape

In [16]:
#test_df.head()

In [17]:
data=[['The musical instruments are very good with nice quality','positve']]


In [18]:
test_df=pd.DataFrame(data,columns=['summary','Result'])
test_df.head()

,summary,Result
0,The musical instruments are very good with nic...,positve


In [19]:

number_of_class_labels=len(train_df['Result'].unique())
number_of_class_labels

3

In [20]:

train_df['Result'].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

In [21]:
Count_Row=train_df.shape[0] 
Count_Col=train_df.shape[1] 

In [22]:
train_df.shape

(10260, 2)

In [23]:

class_prob_df = pd.DataFrame(columns=['Result', 'probability'], index=range(number_of_class_labels))
class_prob_df

,Result,probability
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [24]:
train_df['Result'].value_counts()
    

Positive    9021
Neutral      772
Negative     467
Name: Result, dtype: int64

In [25]:
i=0
for val, cnt in train_df['Result'].value_counts().iteritems():
    print ('value', val, 'was found', cnt, 'times')
    class_prob_df.loc[i].Result = val
    class_prob_df.loc[i].probability = cnt/Count_Row
    i = i +1
    
class_prob_df

value Positive was found 9021 times
value Neutral was found 772 times
value Negative was found 467 times


,Result,probability
0,Positive,0.87924
1,Neutral,0.0752437
2,Negative,0.0455166


In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
#row.summary

In [28]:
train_df['summary'].dtype

dtype('O')

In [29]:

import re
#nltk.download('punkt')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

all_tokens = []

for idx, row in train_df.iterrows():
    for word in word_tokenize(row.summary):
        all_tokens.append(word)
    
print(len(all_tokens), all_tokens)


51306 ['good', 'Jake', 'It', 'Does', 'The', 'Job', 'Well', 'GOOD', 'WINDSCREEN', 'FOR', 'THE', 'MONEY', 'No', 'more', 'pops', 'when', 'I', 'record', 'my', 'vocals', '.', 'The', 'Best', 'Cable', 'Monster', 'Standard', '100', '-', '21', "'", 'Instrument', 'Cable', 'Did', "n't", 'fit', 'my', '1996', 'Fender', 'Strat', '...', 'Great', 'cable', 'Best', 'Instrument', 'Cables', 'On', 'The', 'Market', 'One', 'of', 'the', 'best', 'instrument', 'cables', 'within', 'the', 'brand', 'It', 'works', 'great', 'but', 'I', 'hardly', 'use', 'it', '.', 'HAS', 'TO', 'GET', 'USE', 'TO', 'THE', 'SIZE', 'awesome', 'It', 'works', '!', 'Definitely', 'Not', 'For', 'The', 'Seasoned', 'Piano', 'Player', 'Durable', 'Instrument', 'Cable', 'fender', '18', 'ft.', 'Cali', 'clear', '...', 'So', 'far', 'so', 'good', '.', 'Will', 'revisit', 'at', 'the', '6', 'month', 'mark', ';', ')', 'Add', 'California', 'to', 'the', 'name', 'and', 'I', 'jump', '!', 'Cheap', 'and', 'cool', 'looking', ',', 'good', 'length', 'Fender', '18'

In [30]:
# get only unique tokens
all_tokens_unique = set(all_tokens)
print(len(all_tokens_unique), all_tokens_unique)

6214 {'worhless', '514ce', 'Brilliant', 'tuner-metronome', 'Two', 'alone..nightmare', 'hanging', 'mahogany', 'um', 'Humbuckers', 'ME', 'outstanding', 'Slightly', 'diamond', 'Set-Up', 'wonder', 'Padded', 'ordered', 'HEADPHONES', '6-string', 'ask', 'Workspace', 'MILLION', 'moves', 'Receptacle', 'AWSOME', 'angled', 'volumes', 'sweet', 'Noninvasive', 'WMS', 'Or', 'pickup', 'ukulele', 'DigiVerb', 'split', 'exl', 'satisfied', 'read', 'static', 'PREAMP', 'consistant', 'medium', 'Karaoke', 'connect', 'BRIGHT', 'far', 'Soft', 'SOFT', 'MIC', 'DJ', 'Store', 'butt', 'BS', "Kamakawiwo'ole", 'pricing', 'useless', 'keeps', 'EXCELLENT', 'ease', 'MKII', 'USA', 'Wow', 'Professor', 'bill', 'LENGTHS', 'pics', 'purpose', 'beater', 'WAY', 'engineers', 'Noisy', 'ruged', 'Bench', 'tortex', 'cable', 'floating', 'LP-100', 'doing', 'Could', 'Practically', 'path', 'Still', 'mouse', 'packaging', 'Grip', 'Weight', 'Accept', 'U-Control', 'Guarantee', 'or', 'tried', 'micro', 'DREAM', 'LOCKS', 'chunky', 'Edge', 'IV', 

In [31]:

stop_words = set(stopwords.words('english'))

tokens = [w for w in all_tokens_unique if not w in stop_words]
print(len(tokens), tokens)

tokens1=[]
tokens = [word for word in tokens if word.isalpha()]
print(len(tokens), tokens)

6101 ['worhless', '514ce', 'Brilliant', 'tuner-metronome', 'Two', 'alone..nightmare', 'hanging', 'mahogany', 'um', 'Humbuckers', 'ME', 'outstanding', 'Slightly', 'diamond', 'Set-Up', 'wonder', 'Padded', 'ordered', 'HEADPHONES', '6-string', 'ask', 'Workspace', 'MILLION', 'moves', 'Receptacle', 'AWSOME', 'angled', 'volumes', 'sweet', 'Noninvasive', 'WMS', 'Or', 'pickup', 'ukulele', 'DigiVerb', 'split', 'exl', 'satisfied', 'read', 'static', 'PREAMP', 'consistant', 'medium', 'Karaoke', 'connect', 'BRIGHT', 'far', 'Soft', 'SOFT', 'MIC', 'DJ', 'Store', 'butt', 'BS', "Kamakawiwo'ole", 'pricing', 'useless', 'keeps', 'EXCELLENT', 'ease', 'MKII', 'USA', 'Wow', 'Professor', 'bill', 'LENGTHS', 'pics', 'purpose', 'beater', 'WAY', 'engineers', 'Noisy', 'ruged', 'Bench', 'tortex', 'cable', 'floating', 'LP-100', 'Could', 'Practically', 'path', 'Still', 'mouse', 'packaging', 'Grip', 'Weight', 'Accept', 'U-Control', 'Guarantee', 'tried', 'micro', 'DREAM', 'LOCKS', 'chunky', 'Edge', 'IV', 'Cream', 'F', '

In [32]:
word = ['@', 'rr', '!', '$', '@', 'jfjf', '&','(', ')', ',']
for word in word:
    if word.isalpha():
        print("yes it is alpha: ", word)

yes it is alpha:  rr
yes it is alpha:  jfjf


In [33]:
train_df.values

array([['good', 'Positive'],
       ['Jake', 'Positive'],
       ['It Does The Job Well', 'Positive'],
       ...,
       ['Long life, and for some players, a good economic choice, too',
        'Positive'],
       ['Good for coated.', 'Positive'],
       ['Taylor Made', 'Positive']], dtype=object)

In [34]:
# merge documents for each category
merged_train_df = train_df.groupby('Result')['summary'].apply(' '.join).reset_index()

merged_train_df

,Result,summary
0,Negative,Definitely Not For The Seasoned Piano Player C...
1,Neutral,Didn't fit my 1996 Fender Strat... HAS TO GET ...
2,Positive,good Jake It Does The Job Well GOOD WINDSCREEN...


In [35]:
for idx, row in merged_train_df.iterrows():
    
    temp1_tokens = []
    for word in word_tokenize(row.summary):
        temp1_tokens.append(word)
    
    temp1_tokens = set(temp1_tokens)
         
    temp2_tokens = []
    for word in temp1_tokens:
        if not word in stop_words:
            temp2_tokens.append(word)           
    
    temp3_tokens = []
    for word in temp2_tokens:
        if word.isalpha():
            temp3_tokens.append(word)
            
    print(temp3_tokens)
    temp4_tokens = " ".join(temp3_tokens)
    print(temp4_tokens)
    
    merged_train_df.at[idx, 'summary'] = temp4_tokens
    merged_train_df.at[idx, 'no_of_words_in_category'] = len(temp3_tokens)

['worked', 'break', 'worhless', 'would', 'Straight', 'work', 'cutted', 'Piano', 'MkII', 'Protective', 'junk', 'Slide', 'indepth', 'Two', 'bought', 'applications', 'STRAP', 'last', 'Rocker', 'Useless', 'model', 'light', 'Wide', 'happened', 'knew', 'alternative', 'Waves', 'cant', 'E', 'Its', 'unnecessary', 'CHINA', 'bent', 'fibers', 'thought', 'Beware', 'Seller', 'SINGLE', 'prior', 'Pass', 'reduction', 'Deluxe', 'Unhappy', 'power', 'haaate', 'Snark', 'Include', 'dark', 'electronic', 'GET', 'terrible', 'dissapointed', 'Really', 'problems', 'Get', 'Oh', 'Intonation', 'Item', 'wish', 'Strings', 'Did', 'smell', 'LIGHTWEIGHT', 'TOO', 'cumbersome', 'interface', 'Build', 'control', 'Become', 'Jim', 'CryBaby', 'basses', 'need', 'Cheap', 'EQ', 'love', 'Garbage', 'simple', 'loss', 'knowledge', 'Stars', 'look', 'Noisey', 'Arrival', 'defective', 'Loop', 'mm', 'hisssss', 'Stand', 'Should', 'Crooked', 'POS', 'Headphone', 'pickup', 'ukulele', 'Could', 'Where', 'Cradle', 'Resist', 'Too', 'sounded', 'pic

In [36]:
merged_train_df

,Result,summary,no_of_words_in_category
0,Negative,worked break worhless would Straight work cutt...,933.0
1,Neutral,worked Love Stain MACHINE break gain would Usi...,1128.0
2,Positive,Brilliant Two hanging um Humbuckers ME outstan...,4783.0


In [37]:
# merge to get catgory probability
# merged_train_df
# class_prob_df
merged_train_df = pd.merge(merged_train_df, class_prob_df[['Result', 'probability']], on='Result')

In [38]:
merged_train_df

,Result,summary,no_of_words_in_category,probability
0,Negative,worked break worhless would Straight work cutt...,933.0,0.0455166
1,Neutral,worked Love Stain MACHINE break gain would Usi...,1128.0,0.0752437
2,Positive,Brilliant Two hanging um Humbuckers ME outstan...,4783.0,0.87924


In [39]:
final_df = pd.DataFrame()

row_counter = 0

for idx, row in merged_train_df.iterrows():
    for token in tokens:
        # find the number of occurances of the token in the current category of documents
        no_of_occurances = row.summary.count(token)
        no_of_words_in_category = row.no_of_words_in_category
        no_unique_words_all = len(tokens)
        
        prob_of_token = (no_of_occurances+ 1)/ (no_of_words_in_category+ no_unique_words_all)
        #print(row.class_label, token, no_of_occurances, prob_of_token)
        final_df.at[row_counter, 'Result'] = row.Result
        final_df.at[row_counter, 'token'] = token
        final_df.at[row_counter, 'no_of_occurances'] = no_of_occurances
        final_df.at[row_counter, 'no_of_words_in_category'] = no_of_words_in_category
        final_df.at[row_counter, 'no_unique_words_all'] = no_unique_words_all
        final_df.at[row_counter, 'prob_of_token_category'] = prob_of_token
        
        row_counter = row_counter + 1

In [40]:
final_df

,Result,token,no_of_occurances,no_of_words_in_category,no_unique_words_all,prob_of_token_category
0,Negative,worhless,1.0,933.0,5382.0,0.000317
1,Negative,Brilliant,0.0,933.0,5382.0,0.000158
2,Negative,Two,1.0,933.0,5382.0,0.000317
3,Negative,hanging,0.0,933.0,5382.0,0.000158
4,Negative,mahogany,0.0,933.0,5382.0,0.000158
...,...,...,...,...,...,...
16141,Positive,Dozen,1.0,4783.0,5382.0,0.000197
16142,Positive,Entourage,1.0,4783.0,5382.0,0.000197
16143,Positive,Size,1.0,4783.0,5382.0,0.000197
16144,Positive,Check,1.0,4783.0,5382.0,0.000197


In [41]:
# Calculate P(Category/Document) 
#      = P(Category) * P(Word1/Category) * P(Word2/Category) * P(Word3/Category)

# P(Auto/D6) = P(Auto) * P(Engine/Auto) * P(Noises/Auto) * P(Car/Auto)
for idx, row in test_df.iterrows():
    
    # tokenize & unique words
    temp1_tokens = []
    for word in word_tokenize(row.summary):
        temp1_tokens.append(word)
        #temp1_tokens = set(temp1_tokens)
        
    # remove stop words
    temp2_tokens = []
    for word in temp1_tokens:
        if not word in stop_words:
            temp2_tokens.append(word)
          
    # remove punctuations
    temp3_tokens = []
    for word in temp2_tokens:
        if word.isalpha():
            temp3_tokens.append(word)
            
    #temp4_tokens = " ".join(temp3_tokens)
    #print(temp4_tokens)
    
    prob = 1 
    
    # process for each class_label
    for idx1, row1 in merged_train_df.iterrows():
        print("class: "+ row1.Result)
        for token in temp3_tokens:
            # find the token in final_df for the given category, get the probability
            # row1.class_label & token
        
            print("      : "+ token)  
        
            temp_df = final_df[(final_df['Result'] == row1.Result) & (final_df['token'] == token)]

            # process for exception
            if (temp_df.shape[0] == 0):
                token_prob = 1/(row1.no_of_words_in_category+ no_unique_words_all)
                print("       no token found prob :", token_prob)
                prob = prob * token_prob
            else:
                token_prob = temp_df.get_value(temp_df.index[0],'prob_of_token_category')
                print("       token prob          :", token_prob)
                prob = prob * token_prob

            prob = prob * row1.probability

        col_at = 'prob_'+row1.Result

        test_df.at[idx, col_at] = prob


test_df

class: Negative
      : The
       token prob          : 0.0009501187648456057
      : musical
       token prob          : 0.00015835312747426763
      : instruments
       token prob          : 0.00015835312747426763
      : good
       token prob          : 0.00031670625494853525
      : nice
       token prob          : 0.00031670625494853525
      : quality
       token prob          : 0.00031670625494853525
class: Neutral
      : The
       token prob          : 0.0007680491551459293
      : musical
       token prob          : 0.00015360983102918587
      : instruments
       token prob          : 0.00015360983102918587
      : good
       token prob          : 0.00030721966205837174
      : nice
       token prob          : 0.0004608294930875576
      : quality
       token prob          : 0.00030721966205837174
class: Positive
      : The
       token prob          : 0.0006886374815543532
      : musical
       token prob          : 0.00019675356615838662
      : instruments
 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,summary,Result,prob_Negative,prob_Neutral,prob_Positive
0,The musical instruments are very good with nic...,positve,6.730049e-30,9.627271e-58,4.064090e-79
